In [1]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from tqdm.notebook import tqdm
import pickle

In [2]:
core_top_loc = "../data/fqe_data/Core_top/4/policy_returns.csv"
per_bot_3_loc = "../data/fqe_data/Perimeter_bot_ZN_3/0/policy_returns.csv"
per_top_3_loc = "../data/fqe_data/Perimeter_top_ZN_3/0/policy_returns.csv"

zones = ["Core_top", "Perimeter_bot_ZN_3", "Perimeter_top_ZN_3"]
fqe_dfs = {}
fqe_dfs[zones[0]] = pd.read_csv(core_top_loc, header=None, names=["policy", "score"])
fqe_dfs[zones[1]] = pd.read_csv(per_bot_3_loc, header=None, names=["policy", "score"])
fqe_dfs[zones[2]] = pd.read_csv(per_top_3_loc, header=None, names=["policy", "score"])

eval_data_loc = "../data/1month_eval.csv"
eval_df = pd.read_csv(eval_data_loc, header=None, names=["datetime","policy","zone","energy"])

In [3]:
all_policies = eval_df["policy"].unique()
spearman_corr = {}
for zone in tqdm(zones):
    zone_eval_df = eval_df[eval_df["zone"] == zone]
    valid_policies = fqe_dfs[zone]["policy"].unique()
    invalid_policies = list(set(all_policies) - set(valid_policies))
    for i_policy in invalid_policies:
        zone_eval_df = zone_eval_df[zone_eval_df["policy"]!=i_policy]
    zone_eval_df = zone_eval_df.sort_values(by=["energy"])
    score_list = []
    for i, row in zone_eval_df.iterrows():
        fqe_row = fqe_dfs[zone][fqe_dfs[zone]["policy"] == row["policy"]]
        score_list.append(fqe_row["score"].values[0])

    zone_eval_df["ope_scores"] = score_list
    correlation = spearmanr(zone_eval_df["energy"].values, zone_eval_df["ope_scores"].values)
    spearman_corr[zone] = correlation

  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
spearman_corr

{'Core_top': SpearmanrResult(correlation=0.0012100904529592524, pvalue=0.9851212937033158),
 'Perimeter_bot_ZN_3': SpearmanrResult(correlation=-0.003574268153946839, pvalue=0.9563500556963054),
 'Perimeter_top_ZN_3': SpearmanrResult(correlation=0.011077076185548238, pvalue=0.8652987546899455)}

In [5]:
with open("data/fqe_spearman_corr_12_07_2022.pkl", "wb+") as f:
    pickle.dump(spearman_corr, f)